<a href="https://colab.research.google.com/github/lxb007981/GPT2-Chinese/blob/master/BERT_battleThroughHeaven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Aug  5 11:18:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-coz5oe0h
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-coz5oe0h
     |████████████████████████████████| 3.0MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 43.1MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=817237 sha256=246713a373618427f508401bc7fb4cebad266accb70c75040e8c13eb1313cc14
  Stored in directory: /tmp/pip-ephem-wheel-cache-uu0kigbt/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6135b69c6c48eec62cbe8870bf15b4834531b07564238d7ff92d96de42e391a9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
# model = BertModel.from_pretrained("MODEL_NAME")

In [4]:
!gdown --id 1RuHgKULsTuv4eOBES5QSCCRC-N6E8wx8

Downloading...
From: https://drive.google.com/uc?id=1RuHgKULsTuv4eOBES5QSCCRC-N6E8wx8
To: /content/doupo.txt
16.2MB [00:00, 44.5MB/s]


In [4]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
device = "cuda"
model = RobertaForMaskedLM(config=config)
model.to(device)
model.train()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [6]:
%%time
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./doupo.txt",
    block_size=128,
)

CPU times: user 31.2 s, sys: 126 ms, total: 31.4 s
Wall time: 31.4 s


In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./doupoBERTo",
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [9]:
%%time
trainer.train()

{'loss': 6.229349059104919, 'learning_rate': 3.08868501529052e-05, 'epoch': 0.382262996941896, 'step': 500}
{'loss': 5.521960486412048, 'learning_rate': 1.1773700305810397e-05, 'epoch': 0.764525993883792, 'step': 1000}


CPU times: user 16min 39s, sys: 12min 48s, total: 29min 28s
Wall time: 29min 33s


TrainOutput(global_step=1308, training_loss=5.772519770018551)

In [10]:
trainer.save_model("./doupoBERTo")

In [12]:
!zip -q -r doupoBERT.zip doupoBERTo/

In [13]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
%cp doupoBERT.zip /content/drive/My\ Drive

In [15]:
!gdown --id 1r6vLIk9l49jItfRe-k54Q58PDnVHHFZn

Downloading...
From: https://drive.google.com/uc?id=1r6vLIk9l49jItfRe-k54Q58PDnVHHFZn
To: /content/doupoBERT.zip
312MB [00:01, 261MB/s]


In [16]:
!unzip doupoBERT.zip

Archive:  doupoBERT.zip
   creating: doupoBERTo/
  inflating: doupoBERTo/training_args.bin  
  inflating: doupoBERTo/pytorch_model.bin  
  inflating: doupoBERTo/config.json  


In [18]:
model = RobertaForMaskedLM.from_pretrained("./doupoBERTo")

In [15]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./doupoBERTo",
    tokenizer=tokenizer
)

In [21]:
fill_mask("萧炎[MASK].")

[{'score': 0.3607827425003052,
  'sequence': '[CLS] 萧 炎 ，. [SEP]',
  'token': 8024,
  'token_str': '，'},
 {'score': 0.07170478999614716,
  'sequence': '[CLS] 萧 炎 的. [SEP]',
  'token': 4638,
  'token_str': '的'},
 {'score': 0.02115102857351303,
  'sequence': '[CLS] 萧 炎 这. [SEP]',
  'token': 6821,
  'token_str': '这'},
 {'score': 0.01033884659409523,
  'sequence': '[CLS] 萧 炎 着. [SEP]',
  'token': 4708,
  'token_str': '着'},
 {'score': 0.009479210712015629,
  'sequence': '[CLS] 萧 炎 那. [SEP]',
  'token': 6929,
  'token_str': '那'}]

In [19]:
length = 500
batch_size = 1
nsamples = 6
temperature = 1
topk = 8
topp = 0
repetition_penalty = 1.0
n_ctx = 1024
save_samples = False
fast_pattern = True
save_samples_path = '.'
device = "cuda"
model.to(device)
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [20]:
prefix = '萧炎'

In [21]:
def is_word(word):
    for item in list(word):
        if item not in 'qwertyuiopasdfghjklzxcvbnm':
            return False
    return True


def _is_chinese_char(char):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    cp = ord(char)
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
            (cp >= 0x3400 and cp <= 0x4DBF) or  #
            (cp >= 0x20000 and cp <= 0x2A6DF) or  #
            (cp >= 0x2A700 and cp <= 0x2B73F) or  #
            (cp >= 0x2B740 and cp <= 0x2B81F) or  #
            (cp >= 0x2B820 and cp <= 0x2CEAF) or
            (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
        return True

    return False


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, context, length, n_ctx, tokenizer, temperature=1.0, top_k=30, top_p=0.0, repitition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0)
    generated = context
    with torch.no_grad():
        for _ in trange(length):
            inputs = {'input_ids': generated[0][-(n_ctx - 1):].unsqueeze(0)}
            outputs = model(
                **inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            next_token_logits = outputs[0][0, -1, :]
            for id in set(generated):
                next_token_logits[id] /= repitition_penalty
            next_token_logits = next_token_logits / temperature
            next_token_logits[tokenizer.convert_tokens_to_ids('[UNK]')] = -float('Inf')
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
    return generated.tolist()[0]


def fast_sample_sequence(model, context, length, temperature=1.0, top_k=30, top_p=0.0, device='cpu'):
    inputs = torch.LongTensor(context).view(1, -1).to(device)
    if len(context) > 1:
        _, past = model(inputs[:, :-1], None)[:2]
        prev = inputs[:, -1].view(1, -1)
    else:
        past = None
        prev = inputs
    generate = [] + context
    with torch.no_grad():
        for i in trange(length):
            output = model(prev, past=past)
            output, past = output[:2]
            output = output[-1].squeeze(0) / temperature
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_samples=1)
            generate.append(next_token.item())
            prev = next_token.view(1, 1)
    return generate


# 通过命令行参数--fast_pattern，指定模式
def generate(n_ctx, model, context, length, tokenizer, temperature=1, top_k=0, top_p=0.0, repitition_penalty=1.0, device='cpu',
             is_fast_pattern=False):
    if is_fast_pattern:
        return fast_sample_sequence(model, context, length, temperature=temperature, top_k=top_k, top_p=top_p,
                                    device=device)
    else:
        return sample_sequence(model, context, length, n_ctx, tokenizer=tokenizer, temperature=temperature, top_k=top_k, top_p=top_p,
                               repitition_penalty=repitition_penalty, device=device)



In [22]:
import torch
import torch.nn.functional as F
from tqdm import trange

if length == -1:
    length = n_ctx
if save_samples:
    if not os.path.exists(save_samples_path):
        os.makedirs(save_samples_path)
    samples_file = open(save_samples_path + '/samples.txt', 'w', encoding='utf8')
while True:
    raw_text = prefix
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text))
    generated = 0
    for _ in range(nsamples // batch_size):
        out = generate(
            n_ctx=n_ctx,
            model=model,
            context=context_tokens,
            length=length,
            is_fast_pattern=fast_pattern, tokenizer=tokenizer,
            temperature=temperature, top_k=topk, top_p=topp, repitition_penalty=repetition_penalty, device=device
        )
        for i in range(batch_size):
            generated += 1
            text = tokenizer.convert_ids_to_tokens(out)
            for i, item in enumerate(text[:-1]):  # 确保英文前后有空格
                if is_word(item) and is_word(text[i + 1]):
                    text[i] = item + ' '
            for i, item in enumerate(text):
                if item == '[MASK]':
                    text[i] = ''
                elif item == '[CLS]':
                    text[i] = '\n\n'
                elif item == '[SEP]':
                    text[i] = '\n'
            info = "=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40 + "\n"
            print(info)
            text = ''.join(text).replace('##', '').strip()
            print(text)
            if save_samples:
                samples_file.write(info)
                samples_file.write(text)
                samples_file.write('\n')
                samples_file.write('=' * 90)
                samples_file.write('\n' * 2)
    print("=" * 80)
    if generated == nsamples:
        # close file when finish writing.
        if save_samples:
            samples_file.close()
        break


  3%|▎         | 13/500 [00:00<00:03, 124.55it/s]

======================================== SAMPLE 1 ========================================

萧炎炎炎炎炎炎炎炎炎炎炎炎炎，，，，，，，，，，，，，，，，，，，，点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点着着着着着着着着着着着着头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头头点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着着笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光光笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑笑


  2%|▏         | 12/500 [00:00<00:04, 114.61it/s]

======================================== SAMPLE 2 ========================================

萧炎炎炎炎炎炎炎炎炎炎炎炎是是是是是是是是是是是是，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕怕足足足足足足足足足是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是古古这这这这这这这这这这这这这这这这这这这这这这这这这这这算算他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他他这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不炼这，，，，，，，，，，，，，有有有有有有有有有有有有有有有有有有有有有有有有这这这这这炼炼炼炼炼炼炼不不不不不不不不不不不不不不


  2%|▏         | 12/500 [00:00<00:04, 118.62it/s]

...

======================================== SAMPLE 4 ========================================

萧炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是他他他他他他他他他他他他他他他他他他他他他他样样样到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到到样样样样样样样样样这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这这不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不不


  3%|▎         | 13/500 [00:00<00:03, 124.07it/s]

======================================== SAMPLE 5 ========================================

萧炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎，，，，，，，，，，，，，，，，，，，，，，，，，，点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点点。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了了！！！！！！！！萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你


100%|██████████| 500/500 [00:09<00:00, 51.44it/s]

======================================== SAMPLE 6 ========================================

萧炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧萧，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是是炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎炎
